# Problem

We are a manufactoring company that sell products with credits. For every sell is made a "order".
Once a while the buyers "paid" the credit but is not always the total. We need to associate the pays with orders

# Solution

In [1]:
import scala.annotation.tailrec

import scala.annotation.tailrec

In [2]:
@tailrec
def getMaxValueItems[T, V](
    split: (T, V) => (T, T),
    getValue: T => V,
    minus: (V, V) => V,
    lessZero: V => Boolean,
    equalsZero: V => Boolean
)(
    remain: V,
    items: List[T],
    acum: List[T] = Nil,
): (List[T], List[T]) = {
    if(lessZero(remain) || equalsZero(remain) || items.isEmpty)
        (acum, items)
    else {
        val newRemain = minus(remain, getValue(items.head))
        if(lessZero(newRemain)) {
            val (ac, it) = split(items.head, remain)
            (ac :: acum, it :: items.tail)
        } else getMaxValueItems(split, getValue, minus, lessZero, equalsZero)(
            remain = newRemain,
            items = items.tail,
            acum = items.head :: acum,
        )
    }
}

defined function getMaxValueItems

In [3]:
def associate[I, O, V](
    getMaxValueItem: (V, List[I]) => (List[I], List[I]),
    getValue: O => V
)(
    inputs: List[I],
    outputs: List[O],
): (List[(O, List[I])], List[I]) = {
    @tailrec
    def run[I, O, V](
        getMaxValueItem: (V, List[I]) => (List[I], List[I]),
        getValue: O => V
    )(
        inputs: List[I],
        outputs: List[O],
        acum: List[(O, List[I])]
    ): (List[(O, List[I])], List[I]) =
        if(outputs.isEmpty || inputs.isEmpty)
            (acum, inputs)
        else {
            val (newAcum, newInputs) = getMaxValueItem(getValue(outputs.head), inputs)
            run(getMaxValueItem, getValue)(newInputs, outputs.tail, (outputs.head, newAcum) :: acum)
        }
            
    run(getMaxValueItem, getValue)(inputs, outputs, Nil)   
}

defined function associate

# Application to Buy type

In [4]:
case class Buy(id: Int, price: Long)

val getMaxValueItemsBuy = getMaxValueItems[Buy, Long](
    (a, b) => (a.copy(price = b), a.copy(price = a.price - b)),
    _.price,
    (a, b) => a - b,
    _ < 0,
    _ == 0
)(_, _)

val associateBuy = associate[Buy, Long, Long](getMaxValueItemsBuy, i => i)(_, _)

defined class Buy
getMaxValueItemsBuy: (Long, List[Buy]) => (List[Buy], List[Buy]) = ammonite.$sess.cmd3$Helper$$Lambda$2333/1228605271@4eb06616
associateBuy: (List[Buy], List[Long]) => (List[(Long, List[Buy])], List[Buy]) = ammonite.$sess.cmd3$Helper$$Lambda$2334/45799268@20a34246

In [5]:
val orders = (1 to 15).map(id => Buy(id, id * 1000)).toList
val swaps = List(1000L, 10000l, 100000l)

orders: List[Buy] = List(
  Buy(1, 1000L),
  Buy(2, 2000L),
  Buy(3, 3000L),
  Buy(4, 4000L),
  Buy(5, 5000L),
  Buy(6, 6000L),
  Buy(7, 7000L),
  Buy(8, 8000L),
  Buy(9, 9000L),
  Buy(10, 10000L),
  Buy(11, 11000L),
  Buy(12, 12000L),
  Buy(13, 13000L),
  Buy(14, 14000L),
  Buy(15, 15000L)
)
swaps: List[Long] = List(1000L, 10000L, 100000L)

In [6]:
val swapOrders = associateBuy(orders, swaps)

swapOrders: (List[(Long, List[Buy])], List[Buy]) = (
  List(
    (
      100000L,
      List(
        Buy(15, 6000L),
        Buy(14, 14000L),
        Buy(13, 13000L),
        Buy(12, 12000L),
        Buy(11, 11000L),
        Buy(10, 10000L),
        Buy(9, 9000L),
        Buy(8, 8000L),
        Buy(7, 7000L),
        Buy(6, 6000L),
        Buy(5, 4000L)
      )
    ),
    (10000L, List(Buy(5, 1000L), Buy(4, 4000L), Buy(3, 3000L), Buy(2, 2000L))),
    (1000L, List(Buy(1, 1000L)))
  ),
  List(Buy(15, 9000L))
)